In [5]:
# -*- coding: utf-8 -*-
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import random

def xaver_init(n_inputs, n_outputs, uniform = True):
    if uniform:
        init_range = tf.sqrt(6.0/ (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

# 1) 데이터 및 변수 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

x = tf.placeholder("float", [None, 784])  # mnist data image of shape 28*28=784 (흑백이므로 color를 위한 차원은 없음)
y = tf.placeholder("float", [None, 10])  # 0-9 digits recognition => 10 classes

W1 = tf.get_variable("W1", shape=[784,256], initializer=xaver_init(784, 256)) # "W1" 변수 생성,
W2 = tf.get_variable("W2", shape=[256, 256], initializer=xaver_init(256, 256))
W3 = tf.get_variable("W3", shape=[256, 128], initializer=xaver_init(256, 128))
W4 = tf.get_variable("W4", shape=[128, 128], initializer=xaver_init(128, 128))
W5 = tf.get_variable("W5", shape=[128, 10], initializer=xaver_init(128, 10))

b1 = tf.Variable(tf.zeros([256]))
b2 = tf.Variable(tf.zeros([256]))
b3 = tf.Variable(tf.zeros([128]))
b4 = tf.Variable(tf.zeros([128]))
b5 = tf.Variable(tf.zeros([10]))

# 2) Layer간 연결을 통해 최종 model 정의 (drop out을 이용하여 모델을 학습)
dropout_rate = tf.placeholder("float")

_L1 = tf.nn.relu(tf.add(tf.matmul(x, W1), b1))
L1  = tf.nn.dropout(_L1, dropout_rate)
_L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))
L2  = tf.nn.dropout(_L2, dropout_rate)
_L3 = tf.nn.relu(tf.add(tf.matmul(L2, W3), b3))
L3  = tf.nn.dropout(_L3, dropout_rate)
_L4 = tf.nn.relu(tf.add(tf.matmul(L3, W4), b4))
L4  = tf.nn.dropout(_L4, dropout_rate)

activation = tf.add(tf.matmul(L4, W5), b5) # softmax를 사용하지 않는다.
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(activation, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)  # Gradient Descent
init = tf.global_variables_initializer()

# 4) 학습 실행 (Launch the graph)
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop over all batches
        print("total batch : ", total_batch)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, dropout_rate: 0.7})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, dropout_rate: 0.7}) / total_batch

        # Display logs per epoch step
        if epoch % display_step == 0:
            print ("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))
    print ("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(activation, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print ("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels, dropout_rate: 1}))

    # 5) Test 데이터에서 임의로 1개의 이미지를 선택하여, 정답을 예측하는지 확인해보자.
    import matplotlib.pyplot as plt
    r = random.randint(0, mnist.test.num_examples -1) # 랜덤하게 1개 선택
    # 5-1) label에는 10개의 값을 가진 vector type이 저장되어 있음 [0, 1, 2, ...., 9] 각각의 확률값이 저장
    #      실제 값은 예를들면 [0.1, 0.5, 0.1 .... 0.0] ==> "1"의 확률값이 가장 높음 (index=1)
    print("Label : ", sess.run(tf.argmax(mnist.test.labels[r:r+1], 1))) # 실제 값을 출력 (정답)
    print("Predt : ", sess.run(tf.argmax(activation, 1), {x:mnist.test.images[r:r+1], dropout_rate:1})) # 모델이 예측한 값을 출력
    plt.imshow(mnist.test.images[r:r+1].reshape(28, 28), cmap="Greys", interpolation='nearest')
    plt.show()


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


ValueError: Variable W1 already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-4-005789bda21a>", line 24, in <module>
    W1 = tf.get_variable("W1", shape=[784,256], initializer=xaver_init(784, 256)) # "W1" 변수 생성,
  File "/Users/arnorfati/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/arnorfati/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):
